## Notebook to get Model1 Predictions

In [23]:
# Import Libraries

import pandas as pd
import os
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
from keras.models import load_model

In [2]:
# Read input CSV file with scrapped tweets
path = r'Tweet_All.csv'
tweet_all = pd.read_csv(path)

In [15]:
# dropping duplicates in the dataframe
tweet_all = tweet_all.drop_duplicates(subset=['Text'], keep='first').reset_index(drop=True)
tweet_all.head(3)

,Username_Query,Datetime,Tweet Id,Text
0,kevinhart4real,2021-01-22 02:41:31,1.352446e+18,😂😂😂😂😂😂 https://t.co/S6bufs38c6
1,kevinhart4real,2021-01-21 21:00:42,1.352360e+18,Mood on this beautiful Thursday.... #LiveLoveL...
2,kevinhart4real,2021-01-21 16:47:20,1.352297e+18,Just posted a photo https://t.co/QA4YrxAD39


In [4]:
# loading model
model = load_model('/Users/user/Desktop/Project3/model/model1.h5')
print("Model1 Loaded")
file = open('/Users/user/Desktop/Project3/model/tokenizer1.pickle', 'rb')
tokens = pickle.load(file)
file.close()
print("Tokenizer1 Loaded")

Model1 Loaded
Tokenizer1 Loaded


In [25]:
# read "text" column from all scrapped tweets or enter a custom phrase
text = list(tweet_all.Text)#[0:10]
#text = ['I love spring', 'I have winter']

#### Model Prediction

In [26]:
# Define score labels and model prediction functions
max_length = 20
trunc_type = 'post'
padd_type = 'post'


def decode_sentiment(score):
    label = None
    if score <= 0.5:
        label = 'Negative'
    else:
        label = 'Positive'
    
    return label

def predict(text):
    x_test = pad_sequences(tokens.texts_to_sequences([text]), maxlen=max_length,padding=padd_type,truncating=trunc_type)
    score = model.predict([x_test])
    label = decode_sentiment(score)
    
    return  {"text": text, "label": label, "score": float(score)}

In [27]:
%%time
# Prediction results to DataFrame
df = pd.DataFrame()

for i in text:
    df = df.append(predict(i), ignore_index=True)
    
df['user'] = tweet_all['Username_Query'].values#[0:10]
df['tweet_id'] = tweet_all['Tweet Id'].values#[0:10]
df['datetime'] = tweet_all['Datetime'].values#[0:10]
df.to_csv('model1_predictions.csv', encoding='utf-8', index=False)
df.head(10)

CPU times: user 19min 49s, sys: 1min 24s, total: 21min 13s
Wall time: 16min 22s


,label,score,text,user,tweet_id,datetime
0,Positive,0.515525,😂😂😂😂😂😂 https://t.co/S6bufs38c6,kevinhart4real,1.352446e+18,2021-01-22 02:41:31
1,Positive,0.709734,Mood on this beautiful Thursday.... #LiveLoveL...,kevinhart4real,1.352360e+18,2021-01-21 21:00:42
2,Positive,0.664074,Just posted a photo https://t.co/QA4YrxAD39,kevinhart4real,1.352297e+18,2021-01-21 16:47:20
3,Positive,0.972216,For Celebrity Game Face tonight: \n\nThese cou...,kevinhart4real,1.351701e+18,2021-01-20 01:18:07
4,Positive,0.634962,Haters will say that I’m not tall..... 🤷🏾‍♂️ s...,kevinhart4real,1.351641e+18,2021-01-19 21:20:56
5,Positive,0.599475,Shot by @aspictures &amp; styled by @ashleynor...,kevinhart4real,1.351614e+18,2021-01-19 19:33:52
6,Positive,0.624147,Shot by @aspictures &amp; styled by @ashleynor...,kevinhart4real,1.351332e+18,2021-01-19 00:53:40
7,Positive,0.624147,Shot by @aspictures &amp; styled by @scootscoo...,kevinhart4real,1.351216e+18,2021-01-18 17:13:43
8,Positive,0.614195,Happy Sunday....Shot by @aspictures &amp; styl...,kevinhart4real,1.350888e+18,2021-01-17 19:29:53
9,Positive,0.670969,Shot by the amazing @aspictures &amp; styled b...,kevinhart4real,1.350478e+18,2021-01-16 16:20:10
